In [137]:
import os
import sys
from datetime import datetime
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
sys.path.append(os.path.abspath('../../../scripts/'))
import mdapi_functions as md
from bokeh.io import output_notebook
import bokeh.io as io
from IPython.display import display, Markdown

# Temporal coverage of NHS England datasets

In [138]:
display(Markdown('Last modified: ' + datetime.now().strftime("%d %b %Y")))

Last modified: 16 May 2025

<div style="background-color: rgba(0, 178, 169, 0.3); padding: 5px; border-radius: 5px;"><strong>The availability of NHS England datasets differs according to: when each dataset was established; if the dataset has since been retired; and the time periods permitted to flow to UK LLC.</strong></div>  
<br> 

**Hover over the bar representing each dataset to see its full name and time range.**

In [139]:
output_notebook(hide_banner=True)
dss1 = md.get_md_api_dss()
dss1 = dss1[dss1["source"] == "NHSE"][["table", "collection_start", "collection_end", "table_name"]]
dss1 = dss1[~dss1["collection_start"].isnull()]
dss1 = dss1.set_index(keys="table", drop=False).reindex(["GDPPR","HESOP","HESAPC","HESAPC_MAT","HESAPC_ACP","HESCC","ECDS","HESAE", "COVIDSGSS", "IELISA", "NPEX", "CHESS", "CVS", "CVAR", "CANCER", "DEMOGRAPHICS", "MORTALITY", "MHSDS", "IAPT", "CSDS", "PCM"
]).reset_index(drop=True).iloc[::-1]


def ins_ongoing_date(x):
    if x == "ongoing":
        return datetime.now().date()
    else:
        return datetime.strptime(x, '%m/%Y').date()
dss1["collection_end"] = dss1["collection_end"].apply(lambda x: ins_ongoing_date(x))
dss1["collection_start"] = dss1["collection_start"].apply(lambda x: ins_ongoing_date(x))
dss1.loc[0, "collection_start"] = datetime.strptime("01/1940", '%m/%Y').date()

p = figure(x_axis_type="datetime", y_range=dss1["table"].to_list(), width=600, height=400, toolbar_location=None,
           title="Temporal coverage of NHS England datasets in the UK LLC TRE")
p.hbar(y="table", left='collection_start', right='collection_end', height=0.4, source=ColumnDataSource(dss1), fill_color="#00b289")

hover = HoverTool(tooltips = [("Name","@table_name"), ("Start","@collection_start{%b-%Y}"),("End", "@collection_end{%b-%Y}")], formatters={"@collection_start":"datetime", "@collection_end":"datetime"})
p.add_tools(hover)

p.ygrid.grid_line_color = None
p.xaxis.axis_label = "Dataset Collection Period"

p.outline_line_color = None
show(p)